In [ ]:
import numpy as np

# inicial = 0
# passo = 10
# final = 100
# decimal_place = 3
# tamanho_caract = 5


# for i in np.arange(inicial, final, passo):
#     #print(np.issubdtype(i, np.integer))
#     if np.issubdtype(i, np.integer) is True:   
#         print(f'{round(float(i),decimal_place)}'.zfill(tamanho_caract))
#     else:  
#         print(f'{np.around(float(i),decimal_place)}'.ljust(tamanho_caract,'0'))


def get_properties(filename,wanted_properties):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    properties = {}
    
    # final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' or 'Final defect energy' in line]
    # lines_bottom = lines[final_G_index[0]:]

    parameters = {'S_eV':{'expression':'Entropy','after':0,'field':2},
                  'S_J/molK':{'expression':'Entropy','after':1,'field':1},
                  'Heat_capacity_J/molK':{'expression':'Heat capacity - const volume','after':1,'field':1},
                  'G_eV':{'expression':'Final free energy','after':0,'field':4},
                  'G_kJ/molK':{'expression':'Total free energy (in kJmol-1)','after':1,'field':4},
                  'G_defect_eV':{'expression':'Final defect energy','after':0,'field':4},
                  'frequency_TO':{'expression':'K point      1 =','after':5,'field':4},
                  'frequency_Raman':{'expression':'K point      1 =','after':5,'field':6},
                  'frequency_LO':{'expression':'K point      1 =','after':23,'field':3},
                  **{f'sdlc{i}{j}':{'expression':'Static dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'hfdlc{i}{j}':{'expression':'High frequency dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'C{i}{j}':{'expression':'Elastic Constant Matrix: (Units=GPa)','after':4+i,'field':j} for i in (1,2,3,4,5,6) for j in (1,2,3,4,5,6)},
                  **{f'bulk_modulus_{i}':{'expression':'Mechanical properties :','after':5,'field':4+('Reuss', 'Voigt', 'Hill').index(i)} for i in ('Reuss', 'Voigt', 'Hill')},
                  **{f'lattice_parameter_{i}':{'expression':'Comparison of initial and final structures :','after':6+('a', 'b', 'c').index(i),'field':2} for i in ('a', 'b', 'c')}}

    for element in wanted_properties:
        expression = parameters[element]['expression']
        after = parameters[element]['after']
        field = parameters[element]['field']  
        for line_number,line in enumerate(lines):
            if expression in line: 
                properties[wanted_properties[wanted_properties.index(element)]] = lines[line_number+after].split()[field]
                break
            else:
                properties[wanted_properties[wanted_properties.index(element)]] = '-'

    return(properties)

# Escrevendo as propriedades em um arquivo .csv

import csv

wanted_properties = ['G_eV','lattice_parameter_a','lattice_parameter_b','lattice_parameter_c','bulk_modulus_Reuss',
                     'C11','C12','C44','S_eV','sdlc11','hfdlc11','S_J/molK','Heat_capacity_J/molK']

name_arquivo = 'Li2S_2x2x2_(300.0)'
valor_inicial = 5.5
valor_final = 5.9
passo = 0.01
numero_caracteres = 4
casa_decimal = 2

with open('resultado_optimization.csv','w') as arquivo:

    writer = csv.writer(arquivo)
    writer.writerow(['Temperature']+wanted_properties)
    
    for i in np.arange(valor_inicial,valor_final,passo):
        if np.issubdtype(i, np.integer) is True:   
            #print(f'{round(float(i),2)}'.zfill(4))
            j = f'{round(float(i),casa_decimal)}'.zfill(numero_caracteres) # Determina a quantidade de caracteres da string
            filename = name_arquivo + f'_({j}).out' #f'Li2S_v_Li_New({j}).out' f'Li2S_New({j}).out' 
            fullname = filename
            propriedades = get_properties(fullname,wanted_properties)
            writer.writerow([j] + list(propriedades.values()))
        else:  
            #print(f'{np.around(float(i),2)}'.ljust(4,'0'))
            j = f'{round(float(i),casa_decimal)}'.ljust(numero_caracteres,'0') # Determina a quantidade de caracteres da string
            filename = name_arquivo + f'_({j}).out' #f'Li2S_v_Li_New({j}).out' f'Li2S_New({j}).out' 
            fullname = filename
            propriedades = get_properties(fullname,wanted_properties)
            writer.writerow([j] + list(propriedades.values()))